In [1]:
import pdb
import bugzilla
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
tqdm.pandas()

In [3]:
# this is required, otherwise results will be empty
test_name = '\[sig-network\] pods should successfully create sandboxes by getting pod'

# this is not required but should be used for accurate results
job_name = 'periodic-ci-openshift-release-master-ci-4.6-upgrade-from-stable-4.5-e2e-azure-ovn-upgrade'
# job_name = 'promote-release-openshift-okd-machine-os-content-e2e-aws-4.8'

# bugs linked at timestamps up to this amount of time before today will be returned
max_age = '336h'

url = 'https://search.ci.openshift.org/'
# args = {
#     'type': 'bug+junit',
#     'context': '-1',
#     'name': job_name,
#     'maxAge': max_age,
#     'search': test_name,
# }

# response = requests.post(url, data=args)
# soup = BeautifulSoup(response.content)

In [4]:
# soup.find_all('em')[-1].text

In [5]:
# # check whether this test was a bug-affected test for the current job
# pct_affected_msg = soup.find_all('em')[-1].text

# # get the 2-th element of the message string, and drop the "%" symbol at the end
# pct_affected = float(pct_affected_msg.split()[2][:-1])

# if pct_affected > 0:
#     ret = soup.find('table').find('tbody').find_all('tr')
#     for r in ret:
#         column_values = r.find_all('td')
#         if len(column_values) > 1:
#             # check if bug or junit
#             if column_values[1].text == 'bug':
#                 print(r.td.text)

## Link the bugsss

In [6]:
response = requests.get(
    "https://testgrid.k8s.io/redhat-openshift-informing?id=dashboard-group-bar"
)
html = BeautifulSoup(response.content)
testgrid_script = html.findAll("script")[3]
testgrid_script = testgrid_script.text.split()[5].split(",")
dashboard_names = [x.split(":")[1] for x in testgrid_script if "name" in x]
dashboard_names

['"redhat-assisted-installer"',
 '"redhat-openshift-informing"',
 '"redhat-openshift-ocp-release-4.1-blocking"',
 '"redhat-openshift-ocp-release-4.1-informing"',
 '"redhat-openshift-ocp-release-4.2-blocking"',
 '"redhat-openshift-ocp-release-4.2-informing"',
 '"redhat-openshift-ocp-release-4.3-blocking"',
 '"redhat-openshift-ocp-release-4.3-broken"',
 '"redhat-openshift-ocp-release-4.3-informing"',
 '"redhat-openshift-ocp-release-4.4-blocking"',
 '"redhat-openshift-ocp-release-4.4-broken"',
 '"redhat-openshift-ocp-release-4.4-informing"',
 '"redhat-openshift-ocp-release-4.5-blocking"',
 '"redhat-openshift-ocp-release-4.5-broken"',
 '"redhat-openshift-ocp-release-4.5-informing"',
 '"redhat-openshift-ocp-release-4.6-blocking"',
 '"redhat-openshift-ocp-release-4.6-broken"',
 '"redhat-openshift-ocp-release-4.6-informing"',
 '"redhat-openshift-ocp-release-4.7-blocking"',
 '"redhat-openshift-ocp-release-4.7-broken"',
 '"redhat-openshift-ocp-release-4.7-informing"',
 '"redhat-openshift-ocp-re

### todo: optimize this

In [7]:
associated_bugs = []
for dashboard in tqdm(dashboard_names):
    response = requests.get(f"https://testgrid.k8s.io/{dashboard}/summary")
    job_names = response.json().keys()
    
    for i, job in enumerate(job_names):
        if i >= 2:
            break
        response = requests.get(f"https://testgrid.k8s.io/{dashboard}/table?&show-stale-tests=&tab={job}")
        testnames_df = pd.DataFrame(response.json()["tests"])
        
        if len(testnames_df):

            testnames_with_type = testnames_df['name'][testnames_df['name'].str.contains('\.')]
            testnames_without_type = testnames_df['name'][~testnames_df['name'].str.contains('\.')]

            splitted_testnames = testnames_with_type.str.split('\.', expand=True, n=1)
            if len(splitted_testnames) != 0:
                splitted_testnames = splitted_testnames.loc[:, 1].values.tolist()
            else:
                splitted_testnames = []
            
            # combine both test names with . and test names without .
            testnames = splitted_testnames + testnames_without_type.values.tolist()
            
            args = {
                'type': 'bug+junit',
                'context': '-1',
                'name': job,
                'maxAge': max_age,
            }
            
            for test in testnames:
                args['search'] = test.replace('[', '\[').replace(']', '\]')

                # search for associated bugs
                response = requests.post(url, data=args)
                soup = BeautifulSoup(response.content)
                
                # check whether this test was a bug-affected test for the current job
                em_objects = soup.find_all('em')
                
                for em in em_objects:
                    if 'Found' in em.text:
                        pct_affected = float(em.text.split()[2][:-1])
               
                test_bugs = []
                if pct_affected > 0:
                    ret = soup.find('table').find('tbody').find_all('tr')
                    for r in ret:
                        column_values = r.find_all('td')
                        if len(column_values) > 1:
                            # check if bug or junit
                            if column_values[1].text == 'bug':
                                test_bugs.append(column_values[0].text[1:])
                associated_bugs.append((dashboard, job, test, test_bugs))
    break

  0%|          | 0/40 [00:30<?, ?it/s]


In [8]:
linked_and_associated_bugs = pd.DataFrame(associated_bugs, columns=['dashboard', 'job', 'test_name', 'bug_ids'])
linked_and_associated_bugs.head()

,dashboard,job,test_name,bug_ids
0,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,[sig-node] pods should never transition back t...,"[1916880, 1933760, 1887509, 1943334, 1886920]"
1,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,[sig-arch] Monitor cluster while tests execute,"[1913751, 1918680, 1901734]"
2,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,[sig-network] pods should successfully create ...,"[1908378, 1886922, 1921720, 1882750, 1932811, ..."
3,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,[sig-network] pods should successfully create ...,"[1908378, 1886922, 1882750, 1943334, 1929674, ..."
4,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,[sig-network] pods should successfully create ...,"[1932811, 1882750, 1886922, 1887509, 1908378]"


## Get Bugzilla Details

In [14]:
sample_bugs = linked_and_associated_bugs['bug_ids'][0]
sample_bugs

['1916880', '1933760', '1887509', '1943334', '1886920']

In [18]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

bug = bzapi.getbug(sample_bugs[0])
bug.component

'Node'

In [23]:
fields = [
    'status', 
    'component',
    'components', 
    'platform',
    'priority',
    'product',
    'sub_component',
    'sub_components',
    'summary',
    'tags',
    'target_milestone',
    'target_release',
]

for f in fields:
    print(f'{f} = {getattr(bug, f)}')

status = CLOSED
component = Node
components = ['Node']
platform = Unspecified
priority = unspecified
product = OpenShift Container Platform
sub_component = Kubelet
sub_components = {'Node': ['Kubelet']}
summary = [sig-node] pods should never transition back to pending
tags = []
target_milestone = ---
target_release = ['---']


## todo

combine it into a dataframe s.t. each row has one bugzilla id, and columns are status, component, etc + affected tests/jobs.